In [1]:
import os
import torch
import pandas as pd
import cv2

Loading your yolov5 model - change path to your's model

In [335]:
model = torch.hub.load('ultralytics/yolov5','custom', path='C:/Users/xjack/OneDrive/Pulpit/Magisterka sem 1/best_heavy-object.pt',force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\xjack/.cache\torch\hub\master.zip
YOLOv5  2022-4-10 torch 1.11.0+cu113 CUDA:0 (GeForce GTX 960M, 4096MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 


Enter the path to images folder

In [466]:
img_dir="C:/Users/xjack/OneDrive/Pulpit/Magisterka sem 1/data 2/PWAD/Activity_Picking_heavy_object/Vibek/sequence_1_left"

Make /results directory if it doesn't exist

In [467]:
if (os.path.isdir(img_dir+"/results")==False):
    os.mkdir(img_dir+"/results")

Get the names of all files (images) in specific direction

In [468]:
imgs_names = os.listdir(img_dir)
if "results" in imgs_names:
    del imgs_names[imgs_names.index("results")]

Get full paths of above listed files

In [469]:
imgs_paths=[]
for image in imgs_names:
    imgs_paths.append(img_dir+'/'+image)
    

Now you can store the results in one CSV file (A) or each result in seperate file (B):

A) Store results in ONE pandas dataframe (in format x1,y1,x2,y2...,xcenter,ycenter,object) - pixel cordinates

In [27]:
results_to_csv = pd.DataFrame(columns=["x1","y1","x2","y2","x3","y3","x4","y4","xc","yc","object"])
for i in range(0,len(imgs_names)):
    results=model(imgs_paths[i])
    for p in range(0,len(results.pandas().xyxy[0])):
        results_to_csv = results_to_csv.append(pd.DataFrame({"x1":[results.pandas().xyxy[0].at[p,"xmin"]],"y1":[results.pandas().xyxy[0].at[p,"ymin"]],"x2":[results.pandas().xyxy[0].at[p,"xmin"]],"y2":[results.pandas().xyxy[0].at[p,"ymax"]],"x3":[results.pandas().xyxy[0].at[p,"xmax"]],"y3":[results.pandas().xyxy[0].at[p,"ymin"]],"x4":[results.pandas().xyxy[0].at[p,"xmax"]],"y4":[results.pandas().xyxy[0].at[p,"ymax"]],"xc":[0],"yc":[0],"object":[results.pandas().xyxy[0].at[p,"name"]]},index=[imgs_names[i]]))
    results_to_csv.loc[imgs_names[i],"xc"]=(results_to_csv.loc[imgs_names[i],"x1"]+results_to_csv.loc[imgs_names[i],"x3"])/2
    results_to_csv.loc[imgs_names[i],"yc"]=(results_to_csv.loc[imgs_names[i],"y1"]+results_to_csv.loc[imgs_names[i],"y2"])/2
    if(i==0 or i==100 or i==200 or i==300 or i==400 or i==700 or i==900):
        print(i) #just for checking the progress

0
100
200
300
400
700
900


Change to normalized cordinates (scale to 0-1)

In [28]:
width = cv2.imread(imgs_paths[0]).shape[1] #assuming all images have identical dimensions
height = cv2.imread(imgs_paths[0]).shape[0]
results_to_csv.x1 = results_to_csv.x1/width
results_to_csv.x2 = results_to_csv.x2/width
results_to_csv.x3 = results_to_csv.x3/width
results_to_csv.x4 = results_to_csv.x4/width
results_to_csv.xc = results_to_csv.xc/width
results_to_csv.y1 = results_to_csv.y1/height
results_to_csv.y2 = results_to_csv.y2/height
results_to_csv.y3 = results_to_csv.y3/height
results_to_csv.y4 = results_to_csv.y4/height
results_to_csv.yc = results_to_csv.yc/height

Store the dataframe in file with: 1) image indexes and column names 2) just results 3) results+indexes

In [29]:
results_to_csv.to_csv(path_or_buf = img_dir+"/results/results_with_names_test.csv" )

In [30]:
results_to_csv.to_csv(path_or_buf = img_dir+"/results/results.csv",index=False,header=False )

In [31]:
results_to_csv.to_csv(path_or_buf = img_dir+"/results/results_with index.csv",index=True,header=False )

B.1) Store results in seperate files - pixel cordinates

In [165]:
for i in range(0,len(imgs_names)):
    results_to_csv = pd.DataFrame(columns=["x1","y1","x2","y2","x3","y3","x4","y4","xc","yc","object"])
    results=model(imgs_paths[i])
    for p in range(0,len(results.pandas().xyxy[0])):
        results_to_csv = results_to_csv.append(pd.DataFrame({"x1":[results.pandas().xyxy[0].at[p,"xmin"]],"y1":[results.pandas().xyxy[0].at[p,"ymin"]],"x2":[results.pandas().xyxy[0].at[p,"xmin"]],"y2":[results.pandas().xyxy[0].at[p,"ymax"]],"x3":[results.pandas().xyxy[0].at[p,"xmax"]],"y3":[results.pandas().xyxy[0].at[p,"ymin"]],"x4":[results.pandas().xyxy[0].at[p,"xmax"]],"y4":[results.pandas().xyxy[0].at[p,"ymax"]],"xc":[0],"yc":[0],"object":[results.pandas().xyxy[0].at[p,"name"]]},index=[imgs_names[i]]))
    results_to_csv.loc[imgs_names[i],"xc"]=(results_to_csv.loc[imgs_names[i],"x1"]+results_to_csv.loc[imgs_names[i],"x3"])/2
    results_to_csv.loc[imgs_names[i],"yc"]=(results_to_csv.loc[imgs_names[i],"y1"]+results_to_csv.loc[imgs_names[i],"y2"])/2
    results_to_csv.to_csv(path_or_buf = img_dir+"/results/"+imgs_names[i]+ ".csv",index=False,header=False )
    if(i==0 or i==100 or i==200 or i==300 or i==400 or i==700 or i==900):
        print(i) #just for checking the progress

0
100
200


B.2) Store results in seperate files - normalized cordinates (scale 0-1)

In [470]:
width = cv2.imread(imgs_paths[0]).shape[1] #assuming all images have identical dimensions
height = cv2.imread(imgs_paths[0]).shape[0]
for i in range(0,len(imgs_names)):
    results_to_csv = pd.DataFrame(columns=["x1","y1","x2","y2","x3","y3","x4","y4","xc","yc","object"])
    results=model(imgs_paths[i])
    for p in range(0,len(results.pandas().xyxy[0])):
        results_to_csv = results_to_csv.append(pd.DataFrame({"x1":[results.pandas().xyxy[0].at[p,"xmin"]],"y1":[results.pandas().xyxy[0].at[p,"ymin"]],"x2":[results.pandas().xyxy[0].at[p,"xmin"]],"y2":[results.pandas().xyxy[0].at[p,"ymax"]],"x3":[results.pandas().xyxy[0].at[p,"xmax"]],"y3":[results.pandas().xyxy[0].at[p,"ymin"]],"x4":[results.pandas().xyxy[0].at[p,"xmax"]],"y4":[results.pandas().xyxy[0].at[p,"ymax"]],"xc":[0],"yc":[0],"object":[results.pandas().xyxy[0].at[p,"name"]]},index=[imgs_names[i]]))
    results_to_csv.loc[imgs_names[i],"xc"]=(results_to_csv.loc[imgs_names[i],"x1"]+results_to_csv.loc[imgs_names[i],"x3"])/2
    results_to_csv.loc[imgs_names[i],"yc"]=(results_to_csv.loc[imgs_names[i],"y1"]+results_to_csv.loc[imgs_names[i],"y2"])/2
    results_to_csv.x1 = results_to_csv.x1/width
    results_to_csv.x2 = results_to_csv.x2/width
    results_to_csv.x3 = results_to_csv.x3/width
    results_to_csv.x4 = results_to_csv.x4/width
    results_to_csv.xc = results_to_csv.xc/width
    results_to_csv.y1 = results_to_csv.y1/height
    results_to_csv.y2 = results_to_csv.y2/height
    results_to_csv.y3 = results_to_csv.y3/height
    results_to_csv.y4 = results_to_csv.y4/height
    results_to_csv.yc = results_to_csv.yc/height
    results_to_csv.to_csv(path_or_buf = img_dir+"/results/"+imgs_names[i]+ "_normalized.csv",index=False,header=False )
    if(i==0 or i==100 or i==200 or i==300 or i==400 or i==700 or i==900):
        print(i) #just for checking the progress

0
100
200


Show the bounding box on chosen images - from CSVs from B.1

In [170]:
for img_name in imgs_names[0:40]:
    image=cv2.imread(img_dir+'/'+img_name)
    res=pd.read_csv(img_dir+"/results/"+img_name+".csv",index_col=False,names=["x1","y1","x2","y2","x3","y3","x4","y4","xc","yc","object"])
    for p in range(0,len(res)):
        start_point=(int(res.at[p,"x1"]),int(res.at[p,"y1"]))
        end_point=(int(res.at[p,"x4"]),int(res.at[p,"y4"]))
        image=cv2.rectangle(image,start_point,end_point,(0,255,0),2)
    cv2.imshow(img_name,image)
    cv2.waitKey(0) 
cv2.destroyAllWindows() 